# OnStove notebook

This is the OnStove notebook. The purpose of the notebook is to give users the ability to run through the analysis with example data and it can therefore act as a complement to the publication and read the docs documentation.

The notebook is divded into 4 major parts: 
* **Data processing** - In this part of the analysis different geospatial datasets are read and processed to be used in the analysis. The datasets from this step are saved on the users computer. For future runs on the same area of interest this step can consquently be skipped unless datasets are switched. 
* **Calibration** - In this part the area of interest is calibrated. Raster cells are classified as either urban or rural, the electrification rate in different cells are determined and the rates of differet cooking fuels across settlements are calibrated. The calibrated data is saved in .pkl-file   
* **Model run** - The net-benefit for different stoves are determined in different parts of the study area. Summaries of the results documenting the benefits and costs of each stove type across the entire study area are produced. The results are saved as .pkl-file.  
* **Visualization** - Visulizing and saving different maps related to the results.

Each part of the notebook is divided into several different cells and each cell is described more in depth.

In [18]:
import os, sys, requests, zipfile
import geopandas as gpd
sys.path.append("..")

In [19]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
%autoreload 2
from onstove import OnStove, DataProcessor, RasterLayer, VectorLayer
# from onstove.raster import interpolate
import time

# Downloading example data from the Mendeley database

**Downloading and saving the techno-economic specification file**

In [4]:
tech_specs = r"https://data.mendeley.com/public-files/datasets/7y943f6wf8/files/9129fac1-826b-451d-9c7d-808082e34d23/file_downloaded"
response = requests.get(tech_specs)
open("tech_specs.csv", "wb").write(response.content)

8081

**Downloading and saving the socio-economic specification file**

In [5]:
soc_specs = r"https://data.mendeley.com/public-files/datasets/7y943f6wf8/files/a8ab09e4-e561-4df9-b78e-3ea0718dbcd7/file_downloaded"
response = requests.get(soc_specs)
open("soc_specs.csv", "wb").write(response.content)

1426

**Downloading, saving and unzipping the GIS data**

In [4]:
gis_data = r"https://data.mendeley.com/public-files/datasets/7y943f6wf8/files/58507d24-8996-4857-b528-6543165bbe13/file_downloaded"
response = requests.get(gis_data)
open("gis_data.zip", "wb").write(response.content)

with zipfile.ZipFile("gis_data.zip","r") as zip_ref:
    zip_ref.extractall("gis_data")

# 1. Data processing

## 1.1. Create a data processor

This cell creates your dataprocessor. The dataprocessor is your model and will set the base for your model. OnStove is a raster-based model, hence the resolution becomes important. We specify the resolution when creating the dataprocessor together with the coordinate sytsem. This will ensure that all rasters are resampled to the correct resolution and all datasets (vectors and rasters) are reprojected to the target coordinate system.

In this example we use the world mercator coordinate system (EPSG:3395) and a spatial resolution of 1 sq. km. 

**Note:** This section of the code the user only needs to run once unless the geospatial datasets do not change in between runs. 

In [21]:
start = time.time()

data = DataProcessor(project_crs=3857, cell_size=(1000, 1000))
output_directory = '../example/results'
data.output_directory = output_directory

## 1.2. Add a mask and base 

The mask layer dictates what falls within your area of interest and what is excluded from your analysis. For mask layer we use the administrative boundaries of the country.

For this a function called *add_mask_layer*. *add_mask_layer* takes four inputs:
1. `category`- referencing the cateogry of the layer
2. `name` - referencing the name of the layer
3. `layer_path` - from where to read the data
4. `postgres` - boolean determining indicating whether the data is saved on disc or in a PostgreSQL database. Default is `False`, meaning the dataset is saved on the disc.

Find more information about this function in the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.DataProcessor.add_mask_layer.html)

In [17]:
adm_path = r"../example/gis_data/Administrative/Country_boundaries/Country_boundaries.geojson"
data.add_mask_layer(category='Administrative', name='Country_boundaries', path=adm_path)

## 1.3. Add GIS layers

Similarly, we can add data layers using the `add_layer` method. A layer `name`, `layer_path` and `postgres` conection also need to be provided (the `postgres` conection defaults to `False`). In addition, the following arguments can be passed:
* `category`: this is used to group all datasets into a category in the final output, e.g. `demand` or `supply`. 
* `layer_type`: this argument is required with two possible options `raster` or `vector`, we should pass either one according to the dataset you are adding. 
* `resample`: this defines what resampling method to use when changing the resolution of the raster. The change of resolution happens when the layer gets aligned with the base layer.

Find more information about this function in the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.DataProcessor.add_layer.html).

In the cells below the following datasets are read:

* **Population raster** - a raster layer describing the spatial distribution of peoeple acorss the study area
* **Urban-rural split raster** - a raster layer describing which areas that can be considered urban and rural respectively 
* **Forest raster** - a raster layer describing where forest is availble and where it is not. This is used in order to estimate how far people have to travel in order to collect biomass.
* **Friction raster** - a raster layer describing the walking-only friction across the study area (the time it takes to travel across different cells of the study area by foot). This is used in order to determine the collection time of manure (for biogas) and biomass. 
* **Medium voltage line vector file** - a line vector layer showing the availability of medium voltage lines. This is used in order to estimate which settlements are electrified and which ones are not. Users can also use either transformers or high-voltage lines if available. 
* **Nighttime lights raster** - a raster showing the intensity of anthropogenic light sources. This is used as a proxy for determinig who may have electricity and how does not. 
* **Traveltime raster** - a map showing the time it takes to travel to the closest city with motorized transport. This is used in order to estimate transportation cost of LPG. A user can also provide LPG suppliers (as a point layer) and a motorized friction map to determine the travel time instead of a traveltime map.
* **Livestock rasters** - raster layers showing the headcounts of different livestock (buffaloes, cattles, poultry, goats, pigs and sheeps). This is used in order to assess the availability of manure in different cells of the study area. 
* **Temperature raster** - raster layer describing the temperature across the study area. This is used in order to assess the possibility of using biogas. [If the average temperature decreases below 10 degrees celsius the conversion of small scale biogas digesters reduces significantly making them unviable](https://www.sciencedirect.com/science/article/pii/S2213138821003118).

### 1.3.1. Demographics

In [18]:
pop_path = r"../example/gis_data/Population/Population.tif"
data.add_layer(category='Demographics', name='Population', 
               path=pop_path, layer_type='raster', resample='sum')

urban_path = r"../example/gis_data/Urban/Urban.tif"
data.add_layer(category='Demographics', name='Urban_rural_divide', 
               path=urban_path, layer_type='raster', resample='nearest')

### 1.3.2. Biomass

In [19]:
forest_path = r"../example/gis_data/Forest/Forest.tif"
data.add_layer(category='Biomass', name='Forest', 
               path=forest_path, layer_type='raster', resample='average')

friction_path = r"../example/gis_data/Friction/Friction.tif"
data.add_layer(category='Biomass', name='Friction', path=friction_path, 
               layer_type='raster', resample='average')

### 1.3.3. Electricity

In [20]:
mv_path = r"../example/gis_data/MV lines/MV_lines.geojson"
data.add_layer(category='Electricity', name='MV_lines', 
               path=mv_path, layer_type='vector')

ntl_path = r"../example/gis_data/Night time lights/Night_time_lights.tif"
data.add_layer(category='Electricity', name='Night_time_lights', 
               path=ntl_path, layer_type='raster', resample='average')

### 1.3.4. LPG

In [21]:
lpg_path = r"../example/gis_data/Traveltime/Traveltime.tif"
data.add_layer(category='LPG', name='LPG Traveltime', 
               path=lpg_path, layer_type='raster', resample='average')

### 1.3.5. Biogas

In [22]:
buffaloes = r"../example/gis_data/Livestock/buffaloes/buffaloes.tif"
cattles = r"../example/gis_data/Livestock/cattles/cattles.tif"
poultry = r"../example/gis_data/Livestock/poultry/poultry.tif"
goats = r"../example/gis_data/Livestock/goats/goats.tif"
pigs = r"../example/gis_data/Livestock/pigs/pigs.tif"
sheeps = r"../example/gis_data/Livestock/sheeps/sheeps.tif"

for key, path in {'buffaloes': buffaloes,
                  'cattles': cattles,
                  'poultry': poultry,
                  'goats': goats,
                  'pigs': pigs,
                  'sheeps': sheeps}.items():
    data.add_layer(category='Biogas/Livestock', name=key, path=path,
                   layer_type='raster', resample='nearest', rescale=True)

In [23]:
temperature = r"../example/gis_data/Temperature/Temperature.tif"
data.add_layer(category='Biogas', name='Temperature', path=temperature,
               layer_type='raster', resample='average')
data.layers['Biogas']['Temperature'].save(f'{data.output_directory}/Biogas/Temperature')

Lastly a raster base layer is needed to make every output match its grid and extent. For this, two additional options need to be passed to the `add_layer` method:
* `base_layer`: if `True` the added layer will be considered as the base layer. 
* `resample`: this is the resampling method to be used when resampling this layer to the desired `cell_size` if a `cell_size` is provided.

Find more information about this function in the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.DataProcessor.add_layer.html).

In [24]:
data.add_layer(category='Base', name='Base', path=r"../example/gis_data/Forest/Forest.tif",
               layer_type='raster', base_layer=True, resample='nearest')

## 1.4. Mask reproject and align all required layers

The cell below masks all of the read datasets (rasters and vectors) and categorizes them in different groups. Each dataset that is clipped here is saved in their respective subfolder (e.g. Demographics) under the output_directory specified in [cell 1.1](#1.1.-Create-a-data-processor).

Find more information about this function in the [documenation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.DataProcessor.mask_layers.html).

In [25]:
data.mask_layers(datasets={'Demographics': ['Population', 'Urban_rural_divide'],
                           'Biomass': ['Forest', 'Friction'],
                           'Electricity': ['Night_time_lights', 'MV_lines'],
                           'LPG': ['LPG Traveltime'],
                           'Biogas': ['Temperature']})

Next, all raster datasets are aligned with the base layer selected in [cell 1.2](#1.2.-Add-a-mask-and-base). When aligning the function ensures that the coordinate sysmtem and resolution of all rasters are the same as the user specifies in [cell 1.1](#1.1.-Create-a-data-processor). 

Find more information about this function in the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.DataProcessor.align_layers.html).

In [26]:
data.align_layers(datasets='all')

Lastly, the vector files (apart from the mask layer) are reprojected. This is done using the `reprojected_layers` function. The source coordinate system does not influence the process and the target coordinate system is the same as the one entered by the user in [cell 1.1](#1.1.-Create-a-data-processor). 

Find more information about this function in the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.DataProcessor.reproject_layers.html).

In [27]:
data.reproject_layers(datasets={'Electricity': ['MV_lines']})

In [28]:
end = time.time()

diff = end - start
print('Execution time:', str(str(int(diff//60))) + ' min ' + str(int((diff)%60)) + ' sec')

Execution time: 0 min 41 sec


# 2. Calibration

The calibration step does two things 1) it adds the datasets processed in the previous step to a settlement file that will be used in [step 3](#3.-Model-run) for determining the net-benefit in different settlements and 2) calibrating the file with regards to total population, urban-rural split and electrification rate. 

**Note:** Similar to the data processing step, this step is only needed once unless you change anything in the inputs

## 2.1. Read the model data

The cell below reads the socio-economic specification file. This file is needed as it contains the electrificaiton and urban rates and the actual population in the study area (often the GIS datasets of populations have slighlty outdated values of population). The scenario data is saved in dictionary that can be called upon with `country.specs`

In [5]:
country = OnStove()
path = os.path.join('..', 'example','soc_specs.csv')
country.read_scenario_data(path, delimiter=',')

In [6]:
country.specs["end_year_target"] = 0.8
country.specs["population_start_year"]=30733755
country.specs["population_end_year"]=37294019
country.specs["end_year"]=2030
country.specs["urban_end"]=0.63

In [7]:
country.specs

{'start_year': 2020,
 'inter_year': None,
 'end_year': 2030,
 'inter_year_target': None,
 'end_year_target': 0.8,
 'meals_per_day': 3.0,
 'infra_weight': 2.0,
 'ntl_weight': 1.0,
 'pop_weight': 1.0,
 'discount_rate': 0.03,
 'health_spillover_parameter': 0.112,
 'w_costs': 1.0,
 'w_environment': 1.0,
 'w_health': 1.0,
 'w_spillover': 1.0,
 'w_time': 1.0,
 'country_name': 'Ghana',
 'country_code': 'gh',
 'population_start_year': 30733755,
 'population_end_year': 37294019,
 'urban_start': 0.57,
 'urban_end': 0.63,
 'elec_rate': 0.85,
 'rural_elec_rate': 0.75,
 'urban_elec_rate': 0.93,
 'mort_copd': 10.42,
 'mort_ihd': 50.38,
 'mort_lc': 4.47,
 'mort_alri': 51.71,
 'morb_copd': 1119.55,
 'morb_ihd': 977.35,
 'morb_lc': 4.55,
 'morb_alri': 7160.02,
 'rural_hh_size': 8.0,
 'urban_hh_size': 6.7,
 'mort_stroke': 54.5,
 'morb_stroke': 1004.9,
 'fnrb': 0.113,
 'coi_alri': 39.0,
 'coi_copd': 103.0,
 'coi_ihd': 45.0,
 'coi_lc': 2431.0,
 'coi_stroke': 3970.0,
 'cost_of_carbon_emissions': 51.0,
 'he

## 2.2. Add a population base layer

The population layer is read with `add_layer` and is turned into a dataframe with a column for population using the `population_to_dataframe`. This dataframe will be used to add all the other geospatial datasets and calculate the results on.

Lastly, the population of the dataframe is calibrated to the current population added in the socio-economic specification file using the `calibrate_current_pop` function

Find more information about these function in [here](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.OnStove.population_to_dataframe.html) and [here](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.OnStove.calibrate_current_pop.html).

In [8]:
path = os.path.join(output_directory,'Demographics','Population', 'Population.tif')
country.add_layer(category='Demographics', name='Population', path=path, layer_type='raster', base_layer=True)
country.population_to_dataframe()

## 2.3. Calibrate urban/rural split

The function `calibrate_urban_current_and_future_GHS` is used to classify different rows in of the dataframe produced in the previous step as either urban or rural. This is based on the [GHS dataset](https://ghsl.jrc.ec.europa.eu/download.php?ds=smod). This is important for the baseyear calculations as urban and rural shares of different stoves (and electrification rates) tend to differ.

Find more information about this function in the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.OnStove.calibrate_urban_current_and_future_GHS.html)

In [9]:
ghs_path = output_directory + r"/Demographics/Urban_rural_divide/Urban_rural_divide.tif"
country.calibrate_urban_rural_split(ghs_path)

## 2.4. Add wealth index GIS data and determine the value of time

Next, the Relative Wealth Index (RWI) is added to the dataframe and used in order to estimate the monetary value of time. As the RWI in this example is a csv-file it is not read and added with the other datasets. Instead the function `extract_wealth_index` is used. 

Find more information about this function in the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.OnStove.extract_wealth_index.html)

The extracted RWI is then transformed to value of time using the function `get_value_of_time`. 

Find more information about this function in the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.OnStove.get_value_of_time.html)

In [10]:
wealth_index = r"../example/gis_data/Relative wealth index/GHA_relative_wealth_index.csv"
country.extract_wealth_index(wealth_index, file_type="csv")
country.get_value_of_time()

## 2.5. Calculate distance to electricity infrastructure 

To estimate electricity access rates in different cells across the study area the distance to electricity related infrastructure is important. The following section of the code reads the MV-lines that were processed in the data processing parts and determines the distance between every settlement (raster cell in the population map) and the closest MV-line. This is done using a function called `distance_to_electricity`. 

**Note**: in the following example MV-lines were used for this, but transformers or HV-lines can also be used if available.

Find more information about this function in the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.OnStove.distance_to_electricity.html).

In [11]:
path = os.path.join(output_directory, 'Electricity', 'MV_lines', 'MV_lines.geojson')
mv_lines = VectorLayer('Electricity', 'MV_lines', path=path)
country.distance_to_electricity(mv_lines=mv_lines)

## 2.6. Add nighttime lights data

Nighttime lights are also important for the electrification calibration as it can give you an idication of where electricity consumption is present. The nighttime light map should already have been processed in the [Data processing](#1.-Data-processing) section and therefore, it is only read here and added to the dataframe using the function `raster_to_dataframe`. 

Find out more the function in the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.OnStove.raster_to_dataframe.html)

In [12]:
path = os.path.join(output_directory, 'Electricity', 'Night_time_lights', 'Night_time_lights.tif')
ntl = RasterLayer('Electricity', 'Night_time_lights', path=path)

country.raster_to_dataframe(ntl, name='Night_lights', method='read')

## 2.7. Calibrate current electrified population

Once the distance to electricity related infrastructure (transformers, MV-lines or HV-lines) is determined and the nighttime light intensity is extracted, the current electrification status for different settlements across the study area is determined. This is done in two steps. First, `current_elec` gives a binary classification of the settlements that are likely electrified (1 indicates that at least someone is likely to have access, while 0 indicates that electricity access is not present at all in the cell). 

For more information about the `current_elec` function, please refer to the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.OnStove.current_elec.html)

Once the binary classification is made the electrification rate in the different cells with the value 1 from the current_elec is determined. This is done using the function final_elec. This is an multi-criteria analysis taking into account the population density in each cell, the nighttime light intensity and the distance to either transformers, MV-lines or HV-lines together with weights for each one of these factors supplied by the user in the socio-economic specs file.  

For more information about the `final_elec` function, please refer to the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.OnStove.final_elec.html)

In [13]:
country.current_elec()
country.final_elec()

print('Calibrated grid electrified population fraction:', country.gdf['Elec_pop_calib'].sum() / country.gdf['pop_init_year'].sum())

Calibrated grid electrified population fraction: 0.849720318685382


## 2.8. Read the cooking technologies data

The techno-economic specification file is read. This file does two things: 1) it determines the available fuel/stove combinations used in the analysis and 2) it idicate different techno-economic parameters related to these stoves. If a stove is to be added to the analysis the first step is to add it in this file. The fucntion `read_tech_data` creates a dictionary of technology-classes. This dictionary can be called with `country.techs`

Find out more about this function in the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.OnStove.read_tech_data.html).

In [14]:
path = os.path.join('..', 'example', 'tech_specs.csv')
country.read_tech_data(path, delimiter=',')

## 2.9. Calculating grid added capacity cost

The following cell is used in order to determine the cost of added capacity. For each new household cooking with electricity there is a need to invest in added capacity. The function `get_capacity_cost` determines this cost by taking into account the different costs of power plants and the current electricity mix in the study area (both from the techno-economic specification file). 

Find out more about this function in the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.technology.Electricity.get_capacity_cost.html).

In [15]:
country.techs['Electricity'].get_capacity_cost(country)

## 2.10. Reading GIS data for LPG supply

The LPG traveltime is added to the dataframe. The original dataset gives the time in minutes, so here the values are divided by 60 to ensure that the time worked with is in hours. A factor of 2 is also multiplied with the traveltime to account for the fact that the trips are two-ways. 

In [16]:
path = os.path.join(output_directory, 'LPG', 'LPG Traveltime', 'LPG Traveltime.tif')
lpg = RasterLayer('LPG', 'LPG Traveltime', path = path)

country.techs['LPG'].travel_time = country.raster_to_dataframe(lpg, fill_nodata_method='interpolate', method = 'read') * 2 / 60
#added this
country.raster_to_dataframe(lpg, name='travel_time', method='read', fill_nodata_method='interpolate')

## 2.11. Adding GIS data for Biogas

Biogas potential is dependent on the availability of manure, the availability of manure is in turn dependent on livestock. The cell below determines and adds the livestocks and manure to the dataframe. Apart from adding the livestock datasets to the dataframe, the cell below also takes into account the temperature [(temperatures below 10 degrees celsius render the biogas digesters too inefficient)](https://www.sciencedirect.com/science/article/pii/S2213138821003118) and adds a friction map (walking only friction that is also used for biomass in order to estimate the traveltime for biogas collection). 

Find out more the calibration of livestock in the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.technology.Biogas.recalibrate_livestock).

In [17]:
amin = gpd.read_file(os.path.join(output_directory, 'Administrative', 'Country_boundaries', 'Country_boundaries.geojson'))
buffaloes = os.path.join(output_directory, 'Biogas', 'Livestock', 'buffaloes', 'buffaloes.tif')
cattles = os.path.join(output_directory, 'Biogas', 'Livestock', 'cattles', 'cattles.tif')
poultry =os.path.join(output_directory, 'Biogas', 'Livestock', 'poultry', 'poultry.tif')
goats = os.path.join(output_directory, 'Biogas', 'Livestock', 'goats', 'goats.tif')
pigs = os.path.join(output_directory, 'Biogas', 'Livestock', 'pigs', 'pigs.tif')
sheeps = os.path.join(output_directory, 'Biogas', 'Livestock', 'sheeps', 'sheeps.tif')

country.techs['Biogas'].temperature = os.path.join(output_directory, 'Biogas', 'Temperature', 'Temperature.tif')
country.techs['Biogas'].recalibrate_livestock(country, buffaloes, cattles, poultry, goats, pigs, sheeps)
country.techs['Biogas'].friction_path = os.path.join(output_directory, 'Biomass', 'Friction', 'Friction.tif')

## 2.12. Adding GIS data for Biomass

Biomass is assumed to be collected by the end-users themselves. In this example notebook three biomass stoves are present (traditional biomass, natural draft (Collected Improved biomass) and forced draft. For each one of these stove types the forest and friction layers are added to their respective classes. The forest layer describes where the collection opportunities for biomass exist, while the friction map (only walking-friction) describes is used to determine the least-cost path to the different collection points on the map. The collection point with the lowest traveltime is selected for biomass collection. The user can also give a condition for where collection can be carried out using the `forest_condition` call. The condition restricts certain areas based on canopy cover. In this example no such restriction is given.

Find out more about how biomass is treadted in the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.technology.Biomass.html).

In [18]:
country.techs['Biomass Forced Draft'].friction_path = os.path.join(output_directory, 'Biomass', 'Friction', 'Friction.tif')
country.techs['Biomass Forced Draft'].forest_path = os.path.join(output_directory, 'Biomass', 'Forest', 'Forest.tif')
country.techs['Biomass Forced Draft'].forest_condition = lambda x: x > 0

country.techs['Collected_Improved_Biomass'].friction_path = os.path.join(output_directory, 'Biomass', 'Friction', 'Friction.tif')
country.techs['Collected_Improved_Biomass'].forest_path = os.path.join(output_directory, 'Biomass', 'Forest', 'Forest.tif')
country.techs['Collected_Improved_Biomass'].forest_condition = lambda x: x > 0

country.techs['Collected_Traditional_Biomass'].friction_path = os.path.join(output_directory, 'Biomass', 'Friction', 'Friction.tif')
country.techs['Collected_Traditional_Biomass'].forest_path = os.path.join(output_directory, 'Biomass', 'Forest', 'Forest.tif')
country.techs['Collected_Traditional_Biomass'].forest_condition = lambda x: x > 0

## 2.13. Saving the prepared model inputs

The cell below saves the dataframe created through the calibration-phase as a .pkl-file. This enables users to run several scenarios on the same file without rerunning the previous steps. 

In [19]:
country.output_directory = r""

country.to_pickle("model_inputs.pkl")

end = time.time()

diff = end - start
print('Execution time:', str(str(int(diff//60))) + ' min ' + str(int((diff)%60)) + ' sec')

Execution time: 0 min 57 sec


# 3. Model run

This section runs the main-analysis and produces the results of OnStove. In this section first the cooking options currently in place are determined for urban and rural settlements. This base-line cooking mix is used in order to determine the relative benefits (reduced mortality, reduced morbidiy, time saved and carbon emissions avoided), as well as the relative costs (investments, operation & maintenance, fuel and salvage costs). The benefits and costs for all stove options are combined in every cell to produce the net-benefit (defined as benefits minus costs). The stove with the highest net-benefit is selected for each cell in the study area.

The first two cells in this section reads: 1) the model inputs (output from the calibration phase) and 2) the socio-economic specification file. 

In [22]:
start = time.time()
country = OnStove.read_model("model_inputs.pkl")

In [23]:
path = os.path.join('..', 'example', 'soc_specs.csv')
country.read_scenario_data(path)

In [24]:
country.specs["end_year_target"] = 0.80
country.specs["population_start_year"]=30733755
country.specs["population_end_year"]=37294019
country.specs["end_year"]=2030
country.specs["urban_end"]=0.63

## 3.1 Calculating benefits and costs of each technology and getting the max benefit technology for each cell

The cell below runs the analysis for the technologies specified in the list called `names`. **Note: the technologies in the name list also have to be present in the techno-economic file, otherwise the run will crash.**

The run function calculates all benefits and costs. It also determines the net-benefit and selects the stove with the highest net-benefit. 

For more information on this function, refer to the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.OnStove.run.html)

In [25]:
names = ['Electricity','Traditional_Charcoal' ,'Charcoal ICS' ,
         'LPG', 'Biogas', 'Collected_Traditional_Biomass', 
         'Collected_Improved_Biomass', 'Biomass Forced Draft', 'Pellets Forced Draft']
country.run(technologies=names) 

[Ghana] Calculating clean cooking access
[Ghana] Calculating base fuel properties


In [26]:
from pprint import pprint
pprint(vars(country.base_fuel))

{'bc_intensity': 0,
 'benefits': None,
 'carbon': 0         3604.232313
1         3604.209825
2         3604.237182
3         3604.246520
4         3604.206059
             ...     
130149    3591.255289
130150    3591.248973
130151    3594.679372
130152    3604.042018
130153    3604.043405
Length: 130154, dtype: float64,
 'carbon_intensity': None,
 'ch4_intensity': 0,
 'clean_cooking_access': 0,
 'co2_intensity': 0,
 'co_intensity': 0,
 'costs': 0         237.970897
1         237.658442
2         238.038556
3         238.168287
4         237.606128
             ...    
130149    237.105934
130150    237.018188
130151    236.051473
130152    235.326975
130153    235.346241
Length: 130154, dtype: float64,
 'current_share_rural': 0,
 'current_share_urban': 0,
 'decreased_carbon_costs': 0         183.815848
1         183.814701
2         183.816096
3         183.816573
4         183.814509
             ...    
130149    183.154020
130150    183.153698
130151    183.328648
130152    183.80

In [29]:
country.gdf

,geometry,Pop,IsUrban,pop_init_year,pop_end_year,households_init,households_end,relative_wealth,value_of_time,MV_lines_dist,...,Cattles,Poultry,Goats,Pigs,Sheeps,available_biogas,Temperature,biogas_energy,baseline_costs,year
73048,POINT (-7891.323 786221.876),651.180115,23.0,716.826965,961.398833,106.989099,143.492363,0.799,0.139856,1.000000,...,27.258717,185.502502,14.189289,2.274850,6.556921,0.000000,27.304485,0.000000,787.059505,2021
72774,POINT (-5891.323 787221.876),116.660004,23.0,128.420746,172.236204,19.167275,25.706896,-0.360,0.104286,0.000000,...,27.258717,185.502502,14.189289,2.274850,6.556921,0.000000,27.218756,0.000000,786.892199,2021
73307,POINT (-7891.323 785221.876),122.185150,23.0,134.502884,180.393487,20.075057,26.924401,0.799,0.139856,0.000000,...,27.258717,185.502502,14.189289,2.274850,6.556921,0.000000,27.298342,0.000000,786.365444,2021
72772,POINT (-7891.323 787221.876),797.666077,23.0,878.080444,1177.669864,131.056783,175.771621,0.799,0.139856,0.000000,...,27.258717,185.502502,14.189289,2.274850,6.556921,0.000000,27.329798,0.000000,786.288811,2021
72502,POINT (-5891.323 788221.876),1490.605835,23.0,1640.876953,2200.722440,244.907008,328.466036,-0.299,0.106158,0.000000,...,27.258717,185.502502,14.189289,2.274850,6.556921,0.000000,27.305614,0.000000,785.889247,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128046,POINT (-151891.323 570221.876),162.037369,12.0,142.981277,149.291817,17.872660,18.661477,-0.033,0.114322,2.000000,...,3.777761,935.369873,98.528992,1.895092,54.774563,3189.703369,26.100000,72725.234375,233.139030,0
117429,POINT (-144891.323 635221.876),113.654205,11.0,100.288124,104.714383,12.536016,13.089298,1.052,0.147621,2.000000,...,0.701301,484.682831,53.932892,2.086440,24.863968,1608.471313,25.804787,36673.144531,233.137622,0
128476,POINT (-155891.323 566221.876),8.763803,11.0,7.733152,8.074458,0.966644,1.009307,0.113,0.118802,2.000000,...,3.829517,913.852478,97.535355,1.883185,53.858101,3142.329590,26.074471,71645.109375,233.134542,0
117162,POINT (-144891.323 636221.876),196.384598,11.0,173.289169,180.937361,21.661146,22.617170,0.967,0.145012,2.000000,...,0.701301,484.682831,53.932892,2.086440,24.863968,1608.471313,25.797020,36673.144531,233.119182,0


## 3.2. Printing the summaries

Prints the summary of the results. Note that the results are relative to the base-line, so in some cases costs and benefits can be negative. For each stove in the results the table includes:

* Population using the stove
* Households using the stove
* Total net-benefit generated across the study area due to the use of the stove
* Total number deaths avoided across the study area due to the use of the stove
* The total health costs avoided by using the stove across the study area
* Hours saved by adopting each stove for every household on average
* Opportunity cost due to time saved for each stove type across the study area
* Emissions avoided across the study area due to the use of each stove
* Emission costs avoided across the study area due to the use of each stove
* Total investment costs
* Total fuel costs
* Total O&M costs (operation and maintenance) 
* Total salvage costs

Each summary statistics area also aggregated across the entire study area.

In [ ]:
country.summary()

## 3.3. Saving the results

Saves the full results as .pkl-file. It is important to change the name each time you run a scenario to avoid overwriting your old results.

In [ ]:
country.to_pickle("results.pkl")

# 4. Visualization

The section below gives an indication of what can be visualized using the OnStove results. Note that the actual result files include more information that can also be visualized and extracted for later use. 

## 4.1. Reading the results

First we read in the results produced in [step 3](#3.-Model-run).

In [ ]:
results = OnStove.read_model("results.pkl")

## 4.2. Add a country mask layer

We read the administrative boundaries polygon. This will be used as a backdrop for the maps. 

In [ ]:
output_directory = '../example/results'
path = os.path.join(output_directory,'Administrative','Country_boundaries', 'Country_boundaries.geojson')
mask_layer = VectorLayer('admin', 'adm_0', path=path)
results.mask_layer = mask_layer

## 4.3. Setting the color palette and label names

Setting the color palette for visualization of the maps and graphs. These are dictionaries containing all the stoves included in the analysis and their html-coded colors.

In [ ]:
cmap = {"Biomass ICS (ND)": '#6F4070', "LPG": '#66C5CC', "Biomass": '#FFB6C1',
        "Biomass ICS (FD)": '#af04b3', "Pellets ICS (FD)": '#ef02f5',
        "Charcoal": '#364135', "Charcoal ICS": '#d4bdc5',
        "Biogas": '#73AF48', "Biogas and Biomass ICS (ND)": "#F6029E",
        "Biogas and Biomass ICS (FD)": "#F6029E",
        "Biogas and Pellets ICS (FD)": "#F6029E",
        "Biogas and LPG": "#0F8554", "Biogas and Biomass": "#266AA6",
        "Biogas and Charcoal": "#3B05DF",
        "Biogas and Charcoal ICS": "#3B59DF",
        "Electricity": '#CC503E', "Electricity and Biomass ICS (ND)": "#B497E7",
        "Electricity and Biomass ICS (FD)": "#B497E7",
        "Electricity and Pellets ICS (FD)": "#B497E7",
        "Electricity and LPG": "#E17C05", "Electricity and Biomass": "#FFC107",
        "Electricity and Charcoal ICS": "#660000",
        "Electricity and Biogas": "#f97b72",
        "Electricity and Charcoal": "#FF0000"}

labels = {"Biogas and Electricity": "Electricity and Biogas",
          'Collected Traditional Biomass': 'Biomass',
          'Collected Improved Biomass': 'Biomass ICS (ND)',
          'Traditional Charcoal': 'Charcoal',
          'Biomass Forced Draft': 'Biomass ICS (FD)',
          'Pellets Forced Draft': 'Pellets ICS (FD)'}

## 4.3. Printing map with stoves with highest net-benefit in each settlement

Visualize maps of the stoves with the highest net-benefits together with some important summary results. 

Find out more about this function in the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.OnStove.plot.html).


In [ ]:
results.plot('max_benefit_tech', cmap=cmap, legend_position=(0.9, 0.7),
             title=f'Maximum net-benefit cooking technology', stats_fontsize=9, figsize=(7, 7),
             labels=labels, legend=True, legend_title='Maximum benefit/ncooking technology', dpi=150,
             legend_prop={'title': {'size': 9, 'weight': 'bold'}, 'size': 9},
             rasterized=True, stats=True, stats_position=(0.85, 0.9), save_style=False)

## 4.4. Printing population split in the study area

Prints the population split describing the percentage of population cooking with different stoves.

Find out more about this function in the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.OnStove.plot_split.html).

In [ ]:
results.plot_split(cmap=cmap, labels=labels)

## 4.5. Printing the costs and benefits for each stove category selected

Prints the costs and benefits of each stove type.

Find out more about this function in the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.OnStove.plot_costs_benefits.html).

In [ ]:
results.plot_costs_benefits(labels=labels, height=1.5, width=2)

## 4.6. Printing the range of net-benefit found for each stove per household for each stove selected

Prints the range of net-benefit for each stove included in the results.

Find out more about this function in the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.OnStove.plot_benefit_distribution.html).

In [ ]:
results.plot_benefit_distribution(type='box', groupby='None', cmap=cmap, labels=labels, height=1.5, width=3.5)

## 4.7. Printing the maximum net-benefit across the study area

Visualize the maximum net-benefit spatially. 

In [ ]:
results.plot('maximum_net_benefit', cmap='magma', metric='per_household')

## 4.8. Printing the total costs across the study area

Visualize the total costs spatially.

In [ ]:
results.gdf['total_costs'] = results.gdf['investment_costs'] - results.gdf['salvage_value'] + results.gdf['fuel_costs'] + results.gdf['om_costs']
results.plot('total_costs', cmap='magma', metric='per_household')

## 4.9. Saving selected results

Saving selected results as rasters.

Find out more about this function in the [documentation](https://onstove-documentation.readthedocs.io/en/latest/generated/onstove.onstove.OnStove.to_raster.html).

In [ ]:
results.to_raster('max_benefit_tech', cmap=cmap, labels=labels)
results.to_raster('maximum_net_benefit', metric='per_household')
results.to_raster('maximum_net_benefit', metric='total')
results.to_raster('net_benefit_LPG', metric='per_household')
results.to_raster('net_benefit_Biogas', metric='per_household')
results.to_raster('net_benefit_Collected_Improved_Biomass', metric='per_household')
results.to_raster('total_costs', metric='per_household')
results.to_raster('investment_costs', metric='total')
results.to_raster('deaths_avoided', metric='per_100k')
results.to_raster('time_saved', metric='per_household')
results.to_raster('reduced_emissions', metric='total')
results.to_raster('health_costs_avoided', metric='total')
results.to_raster('Households', metric='sum')